In [1]:
import h5py
import pandas as pd
import numpy as np
import torch

from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm_notebook as tqdm
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [2]:
x_train = h5py.File("./data/X_train.h5", "r")
y_train = pd.read_csv("./data/y_train.csv").values[:, 1].squeeze()

In [3]:
x_test  = h5py.File('./data/X_test_eFVIB85.h5')

In [3]:
data = x_train["features"][:]

In [4]:
data = x_test["features"][:]

In [5]:
features = []
features.append(data[:, :11])
features.append(data[:, 11:].max(1).reshape(-1, 1))
features.append(data[:, 11:].min(1).reshape(-1, 1))
features = np.concatenate(features, 1)

In [9]:
df_train = pd.read_csv('features_temp.csv')
df_test  = pd.read_csv('test.csv')
del df_train['Unnamed: 0']
del df_test['Unnamed: 0']

In [19]:
features = []
for i in tqdm(data[:, 11:]):
    features.append(np.median(np.abs(i)))

In [11]:
forest = RandomForestClassifier(n_estimators = 20)

In [12]:
df_y = pd.read_csv('./data/y_train.csv')

In [13]:
y_train = df_y['label'].values

In [26]:
x_train = df_train.values

In [28]:
forest.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=20,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [31]:
x_test = df_test.values

In [35]:
y_test = forest.predict(x_test)

In [49]:
y_df_test = pd.DataFrame()

In [50]:
y_df_test['id'] = range(len(y_test))

In [51]:
y_df_test['label'] = y_test

In [55]:
y_df_test.to_csv('y_test.csv')